In [1]:
!git clone https://github.com/nguyenduchuyiu/RL-for-5G-Energy-Saving.git

fatal: destination path 'RL-for-5G-Energy-Saving' already exists and is not an empty directory.


In [2]:
%cd RL-for-5G-Energy-Saving

/home/huy/Project/public_1/5GEnergySaving-Round1-public/RL-for-5G-Energy-Saving


In [ ]:
%%writefile config.yaml
training_mode: True
use_gpu: False
checkpoint_path: "models/ppo_model.pth"

actor_lr: 0.00003
critic_lr: 0.0001

qos_grad_coeff: 10.0
energy_grad_coeff: 100.0
cpu_grad_coeff: 1.0
prb_grad_coeff: 1.0
violation_penalty: -50.0
baseline_reward: 5.0

entropy_coef: 0.01
gamma: 0.99
lambda_gae: 0.95
clip_epsilon: 0.2
ppo_epochs: 8
batch_size: 64
buffer_size: 256
n_envs: 4
hidden_dim: 256


Overwriting config.yaml


In [3]:
%%writefile app/train_scenarios/dense_urban.json
{
  "name": "Dense Urban",
  "description": "3GPP Dense urban with macro and micro cells",
  "deploymentScenario": "dense_urban",
  
  "carrierFrequency": 4e9,
  "systemBandwidth": 200e6,
  "layout": "two_layer",
  "isd": 200,
  
  "numSites": 7,
  "numSectors": 3,
  "antennaHeight": 25,
  "cellRadius": 200,
  
  "numUEs": 300,
  "userDistribution": "Uniform/macro",
  "ueSpeed": 3,
  "indoorRatio": 0.8,
  "outdoorSpeed": 30,
  
  "minTxPower": 10,
  "maxTxPower": 46,
  "basePower": 1000,
  "idlePower": 250,
  
  "simTime": 4096,
  "timeStep": 1,
  
  "rsrpServingThreshold": -110,
  "rsrpTargetThreshold": -100,
  "rsrpMeasurementThreshold": -115,
  "dropCallThreshold": 1,
  "latencyThreshold": 50,
  "cpuThreshold": 95,
  "prbThreshold": 95,
  
  "trafficLambda": 20,
  "peakHourMultiplier": 1.3
}

Overwriting app/train_scenarios/dense_urban.json


In [4]:
%%writefile app/train_scenarios/urban_macro.json
    {
  "name": "3GPP Urban Macro",
  "description": "3GPP Urban macro deployment scenario with large cells and continuous coverage",
  "deploymentScenario": "urban_macro",
  
  "carrierFrequency": 2e9,
  "systemBandwidth": 100e6,
  "layout": "hexagonal_grid",
  "isd": 500,
  
  "numSites": 7,
  "numSectors": 3,
  "antennaHeight": 25,
  "cellRadius": 200,
  
  "numUEs": 300,
  "userDistribution": "mixed_outdoor_indoor",
  "ueSpeed": 30,
  "indoorRatio": 0.8,
  "outdoorSpeed": 30,
  
  "minTxPower": 20,
  "maxTxPower": 46,
  "basePower": 1000,
  "idlePower": 250,
  
  "simTime": 0,
  "timeStep": 1,
  
  "rsrpServingThreshold": -110,
  "rsrpTargetThreshold": -100,
  "rsrpMeasurementThreshold": -115,
  "dropCallThreshold": 1,
  "latencyThreshold": 50,
  "cpuThreshold": 95,
  "prbThreshold": 95,
  
  "trafficLambda": 25,
  "peakHourMultiplier": 1.8
}

Overwriting app/train_scenarios/urban_macro.json


In [5]:
%%writefile app/train_scenarios/rural.json
{
  "name": "3GPP Rural Macro",
  "description": "3GPP Rural deployment scenario with wide area coverage for high speed vehicles",
  "deploymentScenario": "rural",
  
  "carrierFrequency": 700e6,
  "systemBandwidth": 20e6,
  "layout": "hexagonal_grid",
  "isd": 1732,
  
  "numSites": 19,
  "numSectors": 3,
  "antennaHeight": 35,
  "cellRadius": 1000,
  
  "numUEs": 100,
  "userDistribution": "mixed_outdoor_indoor",
  "ueSpeed": 120,
  "indoorRatio": 0.5,
  "outdoorSpeed": 120,
  
  "minTxPower": 20,
  "maxTxPower": 46,
  "basePower": 1200,
  "idlePower": 300,
  
  "simTime": 0,
  "timeStep": 1,
  
  "rsrpServingThreshold": -115,
  "rsrpTargetThreshold": -105,
  "rsrpMeasurementThreshold": -120,
  "dropCallThreshold": 2,
  "latencyThreshold": 100,
  "cpuThreshold": 90,
  "prbThreshold": 90,
  
  "trafficLambda": 10,
  "peakHourMultiplier": 1.2
}

Overwriting app/train_scenarios/rural.json


# Train

In [1]:
import yaml
with open('config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    num_envs = config['n_envs']

!python3 app/main_run_scenarios_parallel.py --scenarios-dir app/train_scenarios

/home/huy/anaconda3/envs/mira/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]

=== Running Benchmark Suite (3 scenarios) ===
Scenarios directory: app/train_scenarios/
Using 4 parallel environments per scenario
Loaded scenarios: ['dense_urban', 'rural', 'urban_macro']


--- Scenario 1/3: dense_urban ---
Loaded scenario: Dense Urban
Loaded scenario: Dense Urban
Loaded scenario: Dense Urban
Created 4 parallel environments
  Scenario: dense_urban
  Action dim: 21
  Obs dim: 280
Loaded scenario: Dense Urban
Loaded scenario: Dense Urban
2025-10-29 04:34:48,272 - PPOAgent - INFO - PPO Agent initialized: 21 cells, 300 UEs
2025-10-29 04:34:48,272 - PPOAgent - INFO - State dim: 1444, Action dim: 100
2025-10-29 04:34:48,272 - PPOAgent - INFO -

# Test

In [ ]:
%%writefile config.yaml
training_mode: False
use_gpu: False
checkpoint_path: "models/ppo_model.pth"

actor_lr: 0.00003
critic_lr: 0.0001

qos_grad_coeff: 10.0
energy_grad_coeff: 5.0
cpu_grad_coeff: 1.0
prb_grad_coeff: 1.0
stability_penalty: -10.0
energy_consumption_penalty: -100.0
violation_penalty: -50.0
baseline_reward: 10.0

entropy_coef: 0.01
gamma: 0.99
lambda_gae: 0.95
clip_epsilon: 0.2
ppo_epochs: 8
batch_size: 64
buffer_size: 256
n_envs: 4
hidden_dim: 256

Overwriting config.yaml


In [1]:
!python3 app/main_run_scenarios_python.py --scenarios-dir app/test_scenarios

/home/huy/anaconda3/envs/mira/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]

=== Running Benchmark Suite (3 scenarios) ===


--- Scenario 1/3: dense_urban ---
Loaded scenario: Dense Urban
2025-10-29 03:58:47,829 - PPOAgent - INFO - PPO Agent initialized: 21 cells, 300 UEs
2025-10-29 03:58:47,829 - PPOAgent - INFO - State dim: 1445, Action dim: 100
2025-10-29 03:58:47,829 - PPOAgent - INFO - Device: cpu
2025-10-29 03:58:47,846 - PPOAgent - INFO - Model loaded from models/ppo_model.pth
2025-10-29 03:58:47,846 - PPOAgent - INFO - Loaded checkpoint from models/ppo_model.pth
Created 7 sites for dense_urban scenario
Configuring cells for dense_urban scenario...
Configured 21 cells for dense_urban scenario
Initialized 300 UEs for dense_u